# CCC Python Implementation Profiling

This notebook provides quantitative profiling evidence showing what percentage of CCC (Clustermatch Correlation Coefficient) runtime is attributable to ARI (Adjusted Rand Index) calculations.

**Goal**: Address reviewer feedback requesting profiling data to support the claim that ARI computation represents the main computational bottleneck in CCC.

## Setup

In [1]:
import sys
import os
import cProfile
import pstats
import io
from pstats import SortKey

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add the libs directory to the path
sys.path.insert(0, os.path.abspath('../../libs'))

from ccc.coef.impl import ccc
from ccc.sklearn.metrics import adjusted_rand_index, get_pair_confusion_matrix, get_contingency_matrix

# Set random seed for reproducibility
np.random.seed(42)

print("Setup complete.")

Setup complete.


## Helper Functions

In [2]:
def generate_synthetic_data(n_features: int, n_samples: int, seed: int = 42) -> np.ndarray:
    """Generate synthetic numerical data for benchmarking.
    
    Args:
        n_features: Number of features (rows)
        n_samples: Number of samples (columns)
        seed: Random seed for reproducibility
    
    Returns:
        2D numpy array of shape (n_features, n_samples)
    """
    np.random.seed(seed)
    return np.random.rand(n_features, n_samples)


def profile_ccc(data: np.ndarray, n_jobs: int = 1) -> pstats.Stats:
    """Profile the CCC function and return stats.
    
    Args:
        data: Input data array
        n_jobs: Number of CPU cores to use
    
    Returns:
        pstats.Stats object with profiling results
    """
    profiler = cProfile.Profile()
    profiler.enable()
    
    result = ccc(data, n_jobs=n_jobs)
    
    profiler.disable()
    
    # Create stats object
    stats = pstats.Stats(profiler)
    return stats, result


def extract_function_times(stats: pstats.Stats) -> dict:
    """Extract timing information for key functions from profiling stats.
    
    Args:
        stats: pstats.Stats object
    
    Returns:
        Dictionary with function names and their cumulative times
    """
    # Get all stats
    stats_dict = stats.stats
    
    # Functions we're interested in
    ari_functions = [
        'adjusted_rand_index',
        'get_pair_confusion_matrix', 
        'get_contingency_matrix'
    ]
    
    partitioning_functions = [
        'get_parts',
        'run_quantile_clustering',
        'get_feature_parts'
    ]
    
    other_functions = [
        'cdist_parts_basic',
        'compute_ccc',
        'compute_coef',
        'ccc'
    ]
    
    results = {
        'ari': {},
        'partitioning': {},
        'other': {},
        'total_time': 0.0
    }
    
    for key, value in stats_dict.items():
        filename, line_num, func_name = key
        # value is (ncalls, tottime, cumtime, callers)
        ncalls, tottime, cumtime = value[0], value[2], value[3]
        
        if func_name in ari_functions:
            results['ari'][func_name] = {
                'ncalls': ncalls,
                'tottime': tottime,
                'cumtime': cumtime
            }
        elif func_name in partitioning_functions:
            results['partitioning'][func_name] = {
                'ncalls': ncalls,
                'tottime': tottime,
                'cumtime': cumtime
            }
        elif func_name in other_functions:
            results['other'][func_name] = {
                'ncalls': ncalls,
                'tottime': tottime,
                'cumtime': cumtime
            }
        
        # Track total time from the ccc function
        if func_name == 'ccc' and 'impl.py' in filename:
            results['total_time'] = cumtime
    
    return results


def calculate_percentages(results: dict) -> dict:
    """Calculate percentage of total time for each category.
    
    Args:
        results: Dictionary from extract_function_times
    
    Returns:
        Dictionary with percentages
    """
    total = results['total_time']
    if total == 0:
        return {'ari_pct': 0, 'partitioning_pct': 0, 'other_pct': 0}
    
    # Sum up tottime (exclusive time) for each category
    ari_time = sum(v['tottime'] for v in results['ari'].values())
    part_time = sum(v['tottime'] for v in results['partitioning'].values())
    
    return {
        'ari_time': ari_time,
        'ari_pct': (ari_time / total) * 100,
        'partitioning_time': part_time,
        'partitioning_pct': (part_time / total) * 100,
        'other_time': total - ari_time - part_time,
        'other_pct': ((total - ari_time - part_time) / total) * 100,
        'total_time': total
    }

## Profiling Across Different Workload Sizes

We test with three representative workload sizes:
- **Small**: 10 features x 100 samples (45 pairwise comparisons)
- **Medium**: 50 features x 500 samples (1,225 pairwise comparisons)
- **Large**: 100 features x 1000 samples (4,950 pairwise comparisons)

In [5]:
# Define workload sizes
workloads = [
    {'name': 'Small', 'n_features': 1000, 'n_samples': 1000},
    {'name': 'Medium', 'n_features': 5000, 'n_samples': 1000},
    {'name': 'Large', 'n_features': 10000, 'n_samples': 1000},
]

# Store results for each workload
all_results = []

for workload in workloads:
    print(f"\nProfiling {workload['name']} workload: {workload['n_features']} features x {workload['n_samples']} samples...")
    
    # Generate data
    data = generate_synthetic_data(workload['n_features'], workload['n_samples'])
    n_pairs = (workload['n_features'] * (workload['n_features'] - 1)) // 2
    print(f"  Number of pairwise comparisons: {n_pairs}")
    
    # Profile (using single thread to get clean profiling data)
    stats, ccc_result = profile_ccc(data, n_jobs=1)
    
    # Extract and analyze results
    func_times = extract_function_times(stats)
    percentages = calculate_percentages(func_times)
    
    # Store results
    result = {
        'workload': workload['name'],
        'n_features': workload['n_features'],
        'n_samples': workload['n_samples'],
        'n_pairs': n_pairs,
        **percentages,
        'func_times': func_times
    }
    all_results.append(result)
    
    print(f"  Total time: {percentages['total_time']:.4f}s")
    print(f"  ARI time: {percentages['ari_time']:.4f}s ({percentages['ari_pct']:.1f}%)")
    print(f"  Partitioning time: {percentages['partitioning_time']:.4f}s ({percentages['partitioning_pct']:.1f}%)")

print("\nProfiling complete.")


Profiling Small workload: 1000 features x 1000 samples...
  Number of pairwise comparisons: 499500
  Total time: 200.1082s
  ARI time: 151.2318s (75.6%)
  Partitioning time: 0.4323s (0.2%)

Profiling Medium workload: 5000 features x 1000 samples...
  Number of pairwise comparisons: 12497500


KeyboardInterrupt: 

## Results Summary Table

In [4]:
# Create summary DataFrame
summary_data = []
for r in all_results:
    summary_data.append({
        'Workload': r['workload'],
        'Features': r['n_features'],
        'Samples': r['n_samples'],
        'Pairwise Comparisons': r['n_pairs'],
        'Total Time (s)': f"{r['total_time']:.4f}",
        'ARI Time (s)': f"{r['ari_time']:.4f}",
        'ARI %': f"{r['ari_pct']:.1f}%",
        'Partitioning %': f"{r['partitioning_pct']:.1f}%",
        'Other %': f"{r['other_pct']:.1f}%"
    })

summary_df = pd.DataFrame(summary_data)
print("Runtime Breakdown by Workload Size")
print("=" * 80)
display(summary_df)

Runtime Breakdown by Workload Size


,Workload,Features,Samples,Pairwise Comparisons,Total Time (s),ARI Time (s),ARI %,Partitioning %,Other %
0,Small,10,100,45,0.0178,0.0116,65.0%,6.0%,29.0%
1,Medium,50,500,1225,0.4724,0.3415,72.3%,2.0%,25.8%
2,Large,100,1000,4950,2.0240,1.4965,73.9%,2.1%,23.9%


## Detailed Function-Level Breakdown

Show the number of calls and time spent in each ARI-related function.

In [ ]:
# Show detailed ARI function breakdown for each workload
for r in all_results:
    print(f"\n{r['workload']} Workload - ARI Function Breakdown:")
    print("-" * 60)
    
    ari_data = []
    for func_name, timing in r['func_times']['ari'].items():
        ari_data.append({
            'Function': func_name,
            'Calls': timing['ncalls'],
            'Total Time (s)': f"{timing['tottime']:.6f}",
            'Cumulative Time (s)': f"{timing['cumtime']:.6f}"
        })
    
    if ari_data:
        ari_df = pd.DataFrame(ari_data)
        display(ari_df)
    else:
        print("  No ARI function data captured.")

## Visualizations

In [ ]:
# Set up the figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Colors for categories
colors = {'ARI': '#e74c3c', 'Partitioning': '#3498db', 'Other': '#95a5a6'}

for idx, r in enumerate(all_results):
    ax = axes[idx]
    
    # Data for pie chart
    sizes = [r['ari_pct'], r['partitioning_pct'], r['other_pct']]
    labels = ['ARI', 'Partitioning', 'Other']
    explode = (0.05, 0, 0)  # Explode ARI slice slightly
    
    wedges, texts, autotexts = ax.pie(
        sizes, 
        explode=explode,
        labels=labels,
        colors=[colors[l] for l in labels],
        autopct='%1.1f%%',
        startangle=90,
        pctdistance=0.6
    )
    
    ax.set_title(f"{r['workload']}\n({r['n_features']}x{r['n_samples']}, {r['n_pairs']} pairs)")

plt.suptitle('CCC Runtime Breakdown by Component', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('ccc_runtime_breakdown_pie.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved as 'ccc_runtime_breakdown_pie.png'")

In [ ]:
# Stacked bar chart showing runtime breakdown
fig, ax = plt.subplots(figsize=(10, 6))

workload_names = [r['workload'] for r in all_results]
ari_pcts = [r['ari_pct'] for r in all_results]
part_pcts = [r['partitioning_pct'] for r in all_results]
other_pcts = [r['other_pct'] for r in all_results]

x = np.arange(len(workload_names))
width = 0.6

# Create stacked bars
bars1 = ax.bar(x, ari_pcts, width, label='ARI Computation', color='#e74c3c')
bars2 = ax.bar(x, part_pcts, width, bottom=ari_pcts, label='Partitioning', color='#3498db')
bars3 = ax.bar(x, other_pcts, width, bottom=[a+p for a,p in zip(ari_pcts, part_pcts)], 
               label='Other', color='#95a5a6')

# Add labels on bars
for i, (a, p, o) in enumerate(zip(ari_pcts, part_pcts, other_pcts)):
    ax.text(i, a/2, f'{a:.1f}%', ha='center', va='center', fontweight='bold', color='white')
    ax.text(i, a + p/2, f'{p:.1f}%', ha='center', va='center', fontweight='bold', color='white')

ax.set_ylabel('Percentage of Total Runtime (%)')
ax.set_xlabel('Workload Size')
ax.set_title('CCC Runtime Breakdown by Component', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f"{r['workload']}\n({r['n_features']}x{r['n_samples']})" for r in all_results])
ax.legend(loc='upper right')
ax.set_ylim(0, 105)

plt.tight_layout()
plt.savefig('ccc_runtime_breakdown_bar.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved as 'ccc_runtime_breakdown_bar.png'")

## Analysis and Conclusions

In [ ]:
# Calculate average ARI percentage across all workloads
avg_ari_pct = np.mean([r['ari_pct'] for r in all_results])
min_ari_pct = min([r['ari_pct'] for r in all_results])
max_ari_pct = max([r['ari_pct'] for r in all_results])

print("="*80)
print("SUMMARY: ARI Computation as CCC Bottleneck")
print("="*80)
print(f"\nAcross the tested workloads:")
print(f"  - Average ARI percentage: {avg_ari_pct:.1f}%")
print(f"  - Range: {min_ari_pct:.1f}% to {max_ari_pct:.1f}%")
print(f"\nThese results demonstrate that ARI computation represents the")
print(f"dominant computational component of the CCC algorithm, consuming")
print(f"approximately {avg_ari_pct:.0f}% of the total runtime.")
print("\n" + "="*80)

## Scaling Analysis: Number of ARI Calls

For a dataset with $n$ features and $k$ partitions per feature, CCC performs:
- $\frac{n(n-1)}{2}$ pairwise feature comparisons
- For each comparison: $k^2$ ARI calculations (comparing all partition pairs)

Total ARI calls: $\frac{n(n-1)}{2} \times k^2$

In [ ]:
# Show number of ARI calls for each workload
print("Number of ARI Function Calls by Workload:")
print("-" * 50)

for r in all_results:
    ari_calls = r['func_times']['ari'].get('adjusted_rand_index', {}).get('ncalls', 'N/A')
    n_pairs = r['n_pairs']
    
    print(f"\n{r['workload']} ({r['n_features']}x{r['n_samples']}):")
    print(f"  - Pairwise comparisons: {n_pairs}")
    print(f"  - ARI function calls: {ari_calls}")
    if isinstance(ari_calls, int):
        print(f"  - Avg ARI calls per comparison: {ari_calls/n_pairs:.1f}")

---

## Appendix: Full Profiling Output

For reference, here is the complete cProfile output for the large workload.

In [ ]:
# Re-run profiling for large workload and show full stats
print("Full cProfile output for Large workload (100x1000):")
print("=" * 80)

data = generate_synthetic_data(100, 1000)
stats, _ = profile_ccc(data, n_jobs=1)

# Print top 20 functions by cumulative time
stats.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats(20)